# Machine Learning starterkit
Welcome to the tutorial! First of all we need to import some libraries

In [ ]:
import os
import sys
import warnings
import matplotlib.pyplot as plt
import pickle
import uproot
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn

sys.path.insert(0, '../')
import analysis_utils as au

# avoid pandas warning
warnings.simplefilter(action='ignore', category=FutureWarning)

## A few infos on the libraries

### uproot

uproot is a Python package that provides tools for reading/writing ROOT files using Python and Numpy (does not depend on ROOT) and is primarly intended to stream data into machine learning libraries in Python.

We use uproot for reading and converting ROOT Trees into ***pandas*** **DataFrame**.
For more details: https://github.com/scikit-hep/uproot.

Now we load our data using uproot: signal and background for the training of the models and the unknown data on which we would like to have the model predictions

In [ ]:
signal_tree = uproot.open('/eos/user/a/alicesk/sk2019/data/SignalTree.root')['SignalTable']
background_tree = uproot.open('/eos/user/a/alicesk/sk2019/data/LikeSignTree.root')['BackgroundTable']

data_tree = uproot.open('/eos/user/a/alicesk/sk2019/data/DataTree.root')['DataTable']

In [ ]:
signal_tree.keys()

Then we convert our trees in pandas dataframe

In [ ]:
df_signal = signal_tree.pandas.df()
df_background = background_tree.pandas.df()
df_data = data_tree.pandas.df()

del signal_tree
del background_tree
del data_tree

### Pandas 

Pandas is a library that provides data structures and analysis tools for Pyhton. The two primary data structures of pandas are **Series** (1-dimensional) and **DataFrame** (2-dimensional) and we will work with them.

- **Series** are 1-dimensional ndarray with axis labels.
- **DataFrame** are 2-dimensional tabular data structure with labeled axes (rows and columns).

For more details: https://pandas.pydata.org/pandas-docs/stable/

### Series

In [ ]:
quark_list = ['Up', 'Down', 'Charm', 'Strange', 'Top', 'Bottom']
quark_ser = pd.Series(quark_list)

In [ ]:
quark_ser

In [ ]:
quark_ser.index

In [ ]:
quark_ser.values

We can also define our indexing.

In [ ]:
# data and index must have same lenght (obviously)!
quark_indices = ['q1', 'q2', 'q3', 'q4', 'q5', 'q6']
# Series with custom indexing
quark_ser = pd.Series(data=quark_list, index=quark_indices)

In [ ]:
# we can acces elements by element position


In [ ]:
# or using  index label


It is possible to do operations between series

In [ ]:
ser1 = pd.Series([1,2,3,4,5])
ser2 = pd.Series([5,4,3,2,1])


In [ ]:
ser_sum.values

In [ ]:
ser_product.values

### DataFrame

Can be thought of as a dict-like container for Series objects.

In [ ]:
quark_df = pd.DataFrame(data=quark_list, columns=['names'])
quark_df

We can add more columns to this dataframe

In [ ]:
symbols = ['u', 'd', 'c', 's', 't', 'b']
charge = [2/3, -1/3, 2/3, -1/3, 2/3, -1/3]
generation = [1, 1, 2, 2, 3, 3]

quark_df['symbol'] = symbols
quark_df['charge'] = charge
quark_df['generation'] = generation

quark_df

Columns in a pandas Dataframe can be accessed as dictionaries and return a pandas Series

It is also possible to create a Dataframe from a python dictionary

In [ ]:
#dictionaty with a list as value
dictionary = {'integer': range(0,1000)}
df = pd.DataFrame(dictionary)
df.head()

We can also make operations on the columns and store the result in the Dataframe

In [ ]:
df['pow2'] = df.integer * df.integer
df.head()



One of the most interesting tool of DataFrame is the *query()* method (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html). With this method we can query the DataFrame getting elements which **satisfy a boolean expression**.

In [ ]:
df.query('integer > 100 and pow2 < 14000')

## Let's get back to our data
We can inspect our data easily

In [ ]:
df_signal.head()

It's also quick to make some plots

In [ ]:
#invariant-mass distribution of the signal
minv_sig = df_signal['InvMass'].plot.hist(bins=100, alpha=0.6)

In [ ]:
#invariant-mass distribution of the background (like-sign)
minv_bkg = df_background['InvMass'].plot.hist(bins=100, alpha=0.6)

In [ ]:
#invariant-mass distribution of real data
minv_data = df_data['InvMass'].plot.hist(bins=100, alpha=0.6)

A "trick" to plot more than one distribution is to create a new dataframe with the data to plot in different columns

In [ ]:
#dataframe with invariant-mass of background and signal
df_new = pd.concat([df_background['InvMass'], df_signal['InvMass']], axis=1)
df_new.head()

In [ ]:
minv_compared = df_new.plot.hist(bins=100, alpha=0.6)

## Data preparation
We need to tell at the model what is signal and what is background.

So we add a 'y' column and label signal and background with **y=1** for signal and **y=0** for background. Then we stack togheter signal and background. This will be the reference for the ML model. 

In [ ]:
df_signal['y'] = 1
df_background['y'] = 0

df_ml = pd.concat([df_signal, df_background], axis=0)
df_ml.head()

In [ ]:
df_ml.tail()

### Exploring training variables (features)

- **Exercise:** looking at what we did to plot two distributions togheter compare the 'HypCandPt' disribution between  signal and background

In [ ]:
au.show_solution()

In [ ]:
df_ex = pd.concat([df_background['HypCandPt'], df_signal['HypCandPt']], axis=1)
pt_plot = df_ex.plot.hist(bins=100, alpha=0.6, density=True)
pt_plot.set_yscale('log')

To do this for all the variables we have implemented and utility function `plot_distr`

In [ ]:
#define the variables to plot
columns = ['HypCandPt',
           'TPCnSigmaHe3',
           'V0CosPA',
           'ProngsDCA',
           'He3ProngPvDCA',
           'PiProngPvDCA',
           'He3ProngPvDCAXY',
           'PiProngPvDCAXY',
           'NpidClustersHe3']

au.plot_distr(df_ml, columns, bins=40)

It's interesting to look also at the correlations between the variables. The model can potentially exploit them to perform a better classification. Moreover, there could be some potentially dangerous correlatios as those with the invariant mass of the particle of interest

In [ ]:
au.plot_corr(df_ml, columns, 'signal')

In [ ]:
au.plot_corr(df_ml, columns, 'background')

### Building the Train and Test set
Now we split our data in a training and test set. To do it, we use the `train_test_split` function from the library sklearn <https://scikit-learn.org/stable/>. 

To access the function documentation use Shift+Tab after the first parenthesis

In [ ]:
train_set, test_set, y_train, y_test = sklearn.model_selection.train_test_split(df_ml[columns], df_ml['y'], 
                                                                                test_size=0.5, random_state=42)

In [ ]:
# let's test the lenght of train and test set


## Model training and application
### Example with simple (and weak) model

Let's start with a simple model like the naive bayes <https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html>.

* Train the model

In [ ]:
from sklearn.naive_bayes import GaussianNB
#initialize a the model
nb_model = GaussianNB()

#train the model
nb_model.fit(train_set[columns], y_train)

* Apply the model on the test set and evaluate its performace using the ROC curve <https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics>

In [ ]:
#apply the model on the test set
y_nb = nb_model.predict(test_set[columns])

#evaluate the model performance
nb_score = sklearn.metrics.roc_auc_score(y_test, y_nb)
print(f'ROC_AUC_score: {nb_score:.4f}')

* Now apply the model on the real data that we want to classify and look at the results

In [ ]:
y_pred = nb_model.predict(df_data[columns])

#print some of the predictions
print(y_pred[10:30])

In [ ]:
#add the predictions to the data dataframe
df_data['y_nb'] = y_pred

#invariant-mass distribution after the selections
df_data.query('y_nb>0.5')['InvMass'].plot.hist(bins=100, alpha=0.6)

### A better model: XGBoost

XGBoost is a quite popular library for Boosted Decision Trees <https://xgboost.readthedocs.io/en/latest/>. It is more complex than the previous model and has many parameters to tune.

In [ ]:
#common configuration parameters
xgb_params = {'objective': 'binary:logistic', 
              'tree_method': 'hist',
              'n_jobs': 2,
              'random_state': 42,
              'silent': 1}

### Simple model
We can start with a configuration on hyper-parameters that gives a **simple model**

In [ ]:
hyperparams = {'max_depth': 2,
               'learning_rate': 0.01,
               'n_estimators': 25,
               'subsample': 0.7,
               'colsample_bytree': 0.6}
params = {**xgb_params, **hyperparams}

In [ ]:
#initialize a the model
simple_model = xgb.XGBClassifier(**params)

#train the model
_ = simple_model.fit(train_set[columns], y_train)

#apply the model on the test set
y_test_pred = simple_model.predict_proba(test_set[columns]).T[1]

Now we can compare the model predictions on signal and background between the training set and the test set

In [ ]:
au.plot_output_train_test(simple_model, train_set, y_train, test_set, y_test, columns=columns,
                          figsize=(7,6), log=True, location='upper center')

In [ ]:
#plot the roc curve
au.plot_roc(y_test, y_test_pred)

#evaluate the model performance
model_score = sklearn.metrics.roc_auc_score(y_test, y_test_pred)
print(f'ROC_AUC_score: {model_score:.4f}')

It is also possible to know which variables are more important in the classification using the feature importance given by the XGBoost library

In [ ]:
au.plot_feature_imp(simple_model, imp_list=['gain'])

Model application on data

In [ ]:
#apply the model on the real data
y_pred = simple_model.predict(df_data[columns], output_margin=True)
df_data.eval('score_simple = @y_pred', inplace=True)
df_data.head()

To perform the final classification we still need to choose a threshold value on the BDT score. Particle candidates with a BDT score above this threshold will be considered signal and selected.

To select the threshold we can look at the efficiency vs threshold for example.

In [ ]:
# BDT efficiency
def eff_scan(model, test_set, columns):
    y_pred = model.predict(test_set[columns], output_margin=True)

    test_set.eval('score = @y_pred', inplace=True)
    test_set.eval('y = @y_test', inplace=True)
    min_score = test_set['score'].min()
    max_score = test_set['score'].max()
    threshold = np.linspace(min_score, max_score, 100)
    efficiency = []
    n_sig = sum(test_set['y'])

    for t in threshold:
        df_selected = test_set.query('score>@t')['y']
        sig_selected = np.sum(df_selected)
        efficiency.append(sig_selected / n_sig)

    au.plot_bdt_eff(threshold, efficiency)

In [ ]:
#perform the scan
eff_scan(simple_model, test_set, columns)

In [ ]:
#choose a threshold value
thr = 0.

#invariant-mass distribution after the selections
inv_sel_simple = df_data.query(f'score_simple > {thr}')['InvMass']
_ = inv_sel_simple.plot.hist(bins=100, alpha=0.6)

Now we can fit the distribution and extract the signal

In [ ]:
from scipy.optimize import curve_fit
from scipy import integrate

def fit_invmass(df):
    
    # histogram of the data
    counts, bins = np.histogram(df, bins=40, range=[2.96, 3.05])
    
    # define functions for fitting    
    def gaus_function(x, N, mu, sigma):
        return N * np.exp(-(x-mu)**2/(2*sigma**2))
    
    def pol2_function(x, a, b):
        return (a + x*b)
    
    def fit_function(x, a, b, N, mu, sigma):
        return pol2_function(x, a, b) + gaus_function(x, N, mu, sigma)
    
    # x axis ranges for plots
    x_point = 0.5 * (bins[1:] + bins[:-1])
    r = np.arange(2.96, 3.05, 0.00001)
    r_red = np.arange(2.98, 3.005, 0.0001)
    
    # fit the invariant mass distribution with fit_function() pol2+gauss
    popt, pcov = curve_fit(fit_function, x_point, counts, p0 = [100, -1, 100, 2.99, 0.001])
    
    # plot data
    plt.errorbar(x_point, counts, yerr=np.sqrt(counts), fmt='.', ecolor='k', color='k', elinewidth=1., label='Data')
    
    # plot pol2 and gauss obtained in the fit separately
    plt.plot(r_red, gaus_function(r_red, N=popt[2], mu=popt[3], sigma=popt[4]), label='gaus', color='red')
    plt.plot(r, pol2_function(r, a=popt[0], b=popt[1]), label='pol2', color='green')

    # plot the global fit
    plt.plot(r, fit_function(r, *popt), label='pol2+gauss', color='blue')
    
    # compute significance of the signal
    signal = integrate.quad(gaus_function, 2.98, 3.005, args=(popt[2], popt[3], popt[4]))[0] / 0.00225
    background = integrate.quad(pol2_function, 2.98, 3.005, args=(popt[0], popt[1]))[0] / 0.00225
    print(f'Signal counts: {signal:.0f}')
    print(f'Background counts: {background:.0f}')     
    significance = signal / np.sqrt(signal + background)

    # Add some axis labels
    plt.title(f'significance: {significance:.1f}')
    plt.legend()
    plt.xlabel('$M_{^{3}He+\pi}$ $(\mathrm{GeV/}c^2)$')
    plt.ylabel('counts / 2.25 $\mathrm{MeV/}c^2$')
    plt.show()

In [ ]:
fit_invmass(inv_sel_simple)

### Optimized model
We can optimize the hyper-parameters to have a **more complex model**

In [ ]:
hyperparams = {'max_depth': 13,
               'learning_rate': 0.0982,
               'n_estimators': 181,
               'gamma': 0.4467,
               'min_child_weight': 5.75,
               'subsample': 0.74,
               'colsample_bytree': 0.57}
params = {**xgb_params, **hyperparams}

In [ ]:
#initialize a the model
opt_model = xgb.XGBClassifier(**params)

#train the model
opt_model.fit(train_set[columns], y_train)

#apply the model on the test set
y_test_pred = opt_model.predict_proba(test_set[columns]).T[1]

In [ ]:
#evaluate the model performance
model_score = sklearn.metrics.roc_auc_score(y_test, y_test_pred)
print(f'ROC_AUC_score: {model_score:.4f}')

In [ ]:
#plot the predictions on train and test set
au.plot_output_train_test(opt_model, train_set, y_train, test_set, y_test, columns=columns,
                          figsize=(7,6), log=True, location='upper center')

In [ ]:
#apply the model on the real data
y_pred = opt_model.predict(df_data[columns], output_margin=True)
df_data.eval('score_opt = @y_pred', inplace=True)

#perform the scan
eff_scan(opt_model, test_set, columns)

In [ ]:
#choose a threshold value
thr_opt = 3.24

#invariant-mass distribution after the selections
inv_sel_opt = df_data.query(f'score_opt > {thr_opt}')['InvMass']
df_comp = pd.concat([inv_sel_simple, inv_sel_opt], axis=1)
mass_plot = df_comp.plot.hist(bins=100, alpha=0.6)

In [ ]:
#fit the distribution
fit_invmass(inv_sel_opt)

## Mass Shaping

Let's see what could happens if the model overfit the data. We can train a too complex model

```python
columns = ['V0CosPA',
           'HypCandPt',
           'ProngsDCA',
           'PiProngPvDCAXY',
           'He3ProngPvDCAXY',
           'He3ProngPvDCA',
           'PiProngPvDCA',
           'NpidClustersHe3',
           'TPCnSigmaHe3',
           'PiProngPt',
           'He3ProngPt']

#the model training is very long, we have already trained it
complex_model = pickle.load(open('/eos/user/a/alicesk/sk2019/data/complex_model.sav', 'rb'))

y_pred = complex_model.predict(df_data[columns], output_margin=True)
df_data.eval('score = @y_pred', inplace=True)

y_pred = complex_model.predict(df_background[columns], output_margin=True)
df_background.eval('score = @y_pred', inplace=True)
```

Also the model application on the data is quite slow and we already performed it

In [ ]:
# load data with score given by overtrained model
df_data = pd.read_pickle('/eos/user/a/alicesk/sk2019/data/data_with_score.pkl')

In [ ]:
df_data.query('score>4.5')['InvMass'].plot.hist(bins=40, alpha=0.6, xlim=(2.96,3.05))

In [ ]:
df_selected = df_data.query('score>4.5')['InvMass']
fit_invmass(df_selected)

In [ ]:
# load background data with score given by overtrained model
df_background = pd.read_pickle('data/background_with_score.pkl')
inv_mass_bkg = df_background.query('score>4.5')['InvMass']
inv_mass_bkg.plot.hist(bins=40, alpha=0.6, xlim=(2.96,3.05))

## Final Excercise

Using the query function of pandas perform the "standard" selections

In [ ]:
df_lin_sel = df_data.query('V0CosPA > 0.995')
inv_mass_lin_sel = df_lin_sel['InvMass']

In [ ]:
fit_invmass(inv_mass_lin_sel)